In [144]:
from datetime import datetime
import json

In [145]:
consumer_key = ""
consumer_secret = ""
access_token = "-"
access_token_secret = ""

In [ ]:
from TwitterSearch import *

try:
    ts = TwitterSearch(
        consumer_key = consumer_key,
        consumer_secret = consumer_secret,
        access_token = access_token,
        access_token_secret = access_token_secret
    )
    
    tso = TwitterSearchOrder()
    tso.set_keywords(['Educação','prouni'], or_operator = True)
    tso.set_language('pt')
    
    for tweet in ts.search_tweets_iterable(tso):
        created_at = tweet['created_at']
        user_id = tweet['id_str']
        texto = tweet['text']
        
        # print('created_at: ',created_at,'User_id: ',user_id,'Tweet: ',texto)
    
        with open("tweet.json", "a+") as output:
            data = {
                "created_at": created_at,
                "user_id": user_id,
                "tweet":texto
            }
            output.write("{}\n".format(json.dumps(data)))
except TwitterSearchException as e:
    print(e)

In [21]:
import pandas as pd
df = pd.read_json('tweet.json',lines = True)
df.head(10)

,created_at,user_id,tweet
0,2022-05-23 21:19:21+00:00,1528848077846609920,RT @mylaura_m: Gusttavo Lima defende estado mí...
1,2022-05-23 21:19:20+00:00,1528848076538007552,@ronan_jacques @leanzferdinand @DanPimpao Eu e...
2,2022-05-23 21:19:20+00:00,1528848074805809152,RT @ClimaInfoNews: Os malditos milicos estão q...
3,2022-05-23 21:19:12+00:00,1528848040492257280,RT @vgveiga: #EWF2022 \nO Diretor de Educação ...
4,2022-05-23 21:19:10+00:00,1528848034506936320,"RT @BemAfrontosaaa: gente, n precisa falar cmg..."
5,2022-05-23 21:19:09+00:00,1528848028018294784,"Um conselho, façam um curso sobre educação fin..."
6,2022-05-23 21:19:07+00:00,1528848022628716544,@barbieri_isa eu tirei zero em educação física...
7,2022-05-23 21:19:07+00:00,1528848022586675200,@CAMtulio13 Burra eu vou ser se continuar te d...
8,2022-05-23 21:18:55+00:00,1528847969373540352,RT @CascioCardoso: Não tem educação e provavel...
9,2022-05-23 21:18:54+00:00,1528847965875486720,RT @updatecharts: Dez alunos denunciaram abuso...


In [22]:
#remove duplicatas
df.shape

(5728, 3)

In [23]:
df.drop_duplicates(['tweet'],inplace=True)
df.shape

(2366, 3)

In [27]:
#setando as stopWords
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
#stopwords = "da,de,do,sua,ou,ser,quando,muito,há,nos,já,está,eu,também,só,pelo,pela,até,isso,ela,entre,era,depois,sem,mesmo,aos,ter,seus,quem,nas,me,esse,eles,estão,você,tinha,foram,essa,num,nem,suas,meu,às,minha,têm,numa,pelos,elas,havia,seja,qual,será,nós,tenho,lhe,deles,essas,esses,pelas,este,fosse,dele,tu,te,vocês,vos,lhes,meus,minhas,teu,tua,teus,tuas,nosso,nossa,nossos,nossas,dela,delas,esta,estes,estas,aquele,aquela,aqueles,aquelas,isto,aquilo,estou,está,estamos,estão,estive,esteve,estivemos,estiveram,estava,estávamos,estavam,estivera,estivéramos,esteja,estejamos,estejam,estivesse,estivéssemos,estivessem,estiver,estivermos,estiverem,hei,há,havemos,hão,houve,houvemos,houveram,houvera,houvéramos,haja,hajamos,hajam,houvesse,houvéssemos,houvessem,houver,houvermos,houverem,houverei,houverá,houveremos,houverão,houveria,houveríamos,houveriam,sou,somos,são,era,éramos,eram,fui,foi,fomos,foram,fora,fôramos,seja,sejamos,sejam,fosse,fôssemos,fossem,for,formos,forem,serei,será,seremos,serão,seria,seríamos,seriam,tenho,tem,temos,tém,tinha,tínhamos,tinham,tive,teve,tivemos,tiveram,tivera,tivéramos,tenha,tenhamos,tenham,tivesse,tivéssemos,tivessem,tiver,tivermos,tiverem,terei,terá,teremos,terão,teria,teríamos,teriam"

In [33]:
tweet_tokenizer = TweetTokenizer()
def removeStopWords(instancia):
  palavras_de_parada = set(stopwords.words('portuguese'))
  palavras = [i for i in tweet_tokenizer.tokenize(instancia) if not i in palavras_de_parada]
  return (" ".join(palavras))

In [55]:
import re
def clear_data(instancia):
  specialChars = ".,;_-@!+-/*:%#()?"
  instancia = re.sub(r"http\S+","",instancia).lower().replace('…','').replace('rt','')
  for i in instancia:
    if i in specialChars:
      instancia = instancia.replace(i,'')
  return(instancia)

In [56]:
def preprocessamento(instancia):
  instancia = clear_data(instancia)
  palavras = removeStopWords(instancia)
  return palavras
  
tweets = [preprocessamento(i) for i in list(df['tweet'])]
tweets[:10]

['mylauram gusttavo lima defende estado mínimo pra saúdeeducação alimentação p pobrepq p fazer repasse milionário p prefeituras',
 'ronanjacques leanzferdinand danpimpao entendo preocupação relação escola pq confio pq ge',
 'climainfonews malditos milicos querendo fim gratuidade sus ensino público pago educação moral cívica lembra agr',
 'vgveiga ewf 2022 diretor educação habilidades ocdefr andreas schleicher destacou trabalho brasil direcionamento',
 'bemafrontosaaa gente n precisa falar cmg educação gosta mim fala cmg 😏',
 'conselho façam curso sobre educação financeira vai mudar vida 100',
 'barbieriisa tirei zero educação física isabela',
 'camtulio 13 burra vou ser continuar dando palanque faça tulio vejo tweet pessoa',
 'casciocardoso educação provavelmente instrução porque interpreta coisas tanto quanto poa',
 'updatechas dez alunos denunciaram abusos ambiente familiar após assistirem palestra sobre violência sexual escola goiás']

In [35]:
df['preprocessado'] = tweets
df.head()

,created_at,user_id,tweet,preprocessado
0,2022-05-23 21:19:21+00:00,1528848077846609920,RT @mylaura_m: Gusttavo Lima defende estado mí...,mylauram gusttavo lima defende estado mínimo p...
1,2022-05-23 21:19:20+00:00,1528848076538007552,@ronan_jacques @leanzferdinand @DanPimpao Eu e...,ronanjacques leanzferdinand danpimpao entendo ...
2,2022-05-23 21:19:20+00:00,1528848074805809152,RT @ClimaInfoNews: Os malditos milicos estão q...,climainfonews malditos milicos querendo fim gr...
3,2022-05-23 21:19:12+00:00,1528848040492257280,RT @vgveiga: #EWF2022 \nO Diretor de Educação ...,vgveiga #ewf2022 diretor educação habilidades ...
4,2022-05-23 21:19:10+00:00,1528848034506936320,"RT @BemAfrontosaaa: gente, n precisa falar cmg...",bemafrontosaaa gente n precisa falar cmg educa...


In [39]:
texto = [tweet_tokenizer.tokenize(tweet) for tweet in list(df['preprocessado'])]

In [ ]:
def conta_valores(texto):
  valores = []
  for i in texto:
    valores += i
  
  res = {i:valores.count(i) for i in valores}
  return res

contagem_palavras = conta_valores(texto)
print(contagem_palavras)


In [ ]:
print(contagem_palavras)

In [43]:
dataframe_contagem = pd.DataFrame.from_dict(contagem_palavras,orient='index', columns=["quantidade"]).rename_axis("palavras").reset_index().sort_values('quantidade', ascending=False)
dataframe_contagem

,palavras,quantidade
35,educação,1547
40,?,246
401,"""",201
6,pra,201
77,física,177
...,...,...
4215,calado,1
4212,#opinião,1
4208,falácia,1
4205,6693860,1


In [44]:
dataframe_contagem.to_csv('arquivos/out.csv',sep=";") 
